In [1]:
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
tqdm.pandas()

## Nettoyage du DF

In [27]:
# Open du fichier JSON
with open('spots_grimpe.json', 'r') as file:
    data = json.load(file)

In [28]:
# JSON TO DF
df = pd.DataFrame(data['data'])

In [18]:
# Suppression des colonnes
df = df.drop(columns= ["ref_topo", "equipement", "topo_num"])

In [19]:
# Supression du tiret situé à la fin
def sup_tiret(row):
    return row.rstrip("-")


df["type_escalade"] = df["type_escalade"].apply(sup_tiret)
df["exposition"] = df["exposition"].apply(sup_tiret)
df["public"] = df["public"].apply(sup_tiret)

In [6]:
# Création de liste à la place d'une chaîne de caractéres
def split_list(row):
    return row.split("-")

df["type_escalade"] = df["type_escalade"].apply(split_list)
df["exposition"] = df["exposition"].apply(split_list)
df["public"] = df["public"].apply(split_list)
df["saison"] = df["saison"].apply(split_list)

In [20]:
# Remplacer "hautniveau" par "haut niveau" dans les listes
df['public'] = df['public'].apply(lambda lst: ['haut niveau' if x == 'hautniveau' else x for x in lst])

In [21]:
# Dictionnaire de mappage
mapping_orientation = {
    "S": "Sud",
    "N": "Nord",
    "W": "Ouest",
    "E": "Est",
    "SW": "Sud-Ouest",
    "SE": "Sud-Est",
    "NW": "Nord-Ouest",
    "NE": "Nord-Est"
}

# Appliquer le mappage aux listes dans la colonne "exposition"
df["exposition"] = df["exposition"].apply(lambda lst: [mapping_orientation.get(el, el) for el in lst])

In [22]:
# Dictionnaire de mappage
mapping_mois = {
    "1": "Janvier",
    "2": "Février",
    "3": "Mars",
    "4": "Avril",
    "5": "Mai",
    "6": "Juin",
    "7": "Juillet",
    "8": "Août",
    "9": "Septembre",
    "10": "Octobre",
    "11": "Novembre",
    "12": "Decembre"

}

# Appliquer le mappage aux listes dans la colonne "saison"
df["saison"] = df["saison"].apply(lambda lst: [mapping_mois.get(el, el) for el in lst])

In [23]:
# Suppression des () et des - puis remplacer les espaces par des tirets pour l'url
df['url'] = df['nom'].str.replace('(', '').str.replace(')', '').str.replace(' - ', '-').str.replace(' ', '-')

/var/folders/fk/s9b7316s21b6krxxntv4pzzm0000gn/T/ipykernel_6263/3093565319.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['url'] = df['nom'].str.replace('(', '').str.replace(')', '').str.replace(' - ', '-').str.replace(' ', '-')


## Scrapping des urls des images

In [11]:
# Config BS4
url = "https://climbingaway.fr/fr/site-escalade/"
navigator = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1)'

In [13]:
# Scrap de la class li de la page
def scrap_url_img(row):
    img_url = "https://climbingaway.fr/"
    base_url = "https://climbingaway.fr/fr/site-escalade/"
    page_url = row['url']  # Assurez-vous que cette colonne existe dans votre DataFrame
    
    # Envoie une requête pour récupérer la page HTML
    response = requests.get(base_url + page_url, headers={'User-Agent': navigator})
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Trouve la première balise <li> avec la classe 'minie_galerie_li'
    class_li = soup.find_all('li', {"class": "minie_galerie_li"})
    
    if class_li:
        # Extraire l'URL de l'image
        url_img = class_li[0].find('img').get('src')
        
        # Nettoyer l'URL en enlevant '/..' au début
        cleaned_url = url_img.lstrip('/..')
        
        # Retourner l'URL complète
        return img_url + cleaned_url
    return None  # Si aucune image n'est trouvée

df['img_url'] = df.progress_apply(scrap_url_img, axis=1)

In [24]:
# Export CSV
df.to_csv("spots_grimpe_folium.csv")

## Analyse

In [25]:
# Import CSV
df = pd.read_csv("spots_grimpecsv")